In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pdp
%matplotlib inline

In [45]:
df = pd.read_csv('glasses_data.csv')

In [46]:
pdp.ProfileReport(df)

Number of variables,18
Number of observations,136387
Total Missing (%),0.0%
Total size in memory,18.7 MiB
Average record size in memory,144.0 B
Numeric,9
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [48]:
# Cleaning data
df.drop_duplicates(inplace=True)
df.drop(columns=['address', 'first_purchase', 'last_purchase', 'patient_name', 'purchase_time', 'id', 'lat', 'lon'], axis=1, inplace=True)
pdp.ProfileReport(df)

Number of variables,11
Number of observations,50997
Total Missing (%),0.0%
Total size in memory,4.3 MiB
Average record size in memory,88.0 B
Numeric,7
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [49]:
gender = pd.get_dummies(df['gender'], drop_first=True)
exam_type = pd.get_dummies(df['appt_type'], drop_first=True)
insurance = pd.get_dummies(df['insurance'], drop_first=True)

In [50]:
# Removing original categorical data columns.
df.drop(columns=['gender', 'appt_type', 'insurance', 'used_ins'], axis=1, inplace=True)

In [51]:
df.head()

,avg_dollar,age,total_paid,product_id,price,buying_pattern
0,51000,53,51000,10,30000,1
1,51000,53,51000,15,25900,1
2,51000,53,51000,18,0,1
3,51000,53,51000,21,7500,1
4,51000,53,51000,1,8500,1


In [53]:
df2 = pd.concat([df, gender, exam_type, insurance], join='inner' ,axis=1)
df2.head()

,avg_dollar,age,total_paid,product_id,price,buying_pattern,male,Glasses,Health,None,Poor,Standard
0,51000,53,51000,10,30000,1,1,1,0,0,0,1
1,51000,53,51000,15,25900,1,1,1,0,0,0,1
2,51000,53,51000,18,0,1,1,1,0,0,0,1
3,51000,53,51000,21,7500,1,1,1,0,0,0,1
4,51000,53,51000,1,8500,1,1,1,0,0,0,1


In [56]:
# First testing to find buying patterns (1 lowest, 5 highest)
X = df2.drop('buying_pattern', axis=1)
y = df2['buying_pattern']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [71]:
from sklearn.tree import DecisionTreeClassifier  # Single tree, accuracy with defaults same as random forest.
from sklearn.metrics import classification_report, confusion_matrix

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

predictions = dtree.predict(X_test)

print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))  # Mid 90's accuracy for testing buying groups.

[[ 777   60   17    1    0]
 [  93 2438  184   22    1]
 [  50  152 4515   79    4]
 [  10   33   95 4751   41]
 [   0    3    8   51 1915]]


              precision    recall  f1-score   support

           1       0.84      0.91      0.87       855
           2       0.91      0.89      0.90      2738
           3       0.94      0.94      0.94      4800
           4       0.97      0.96      0.97      4930
           5       0.98      0.97      0.97      1977

   micro avg       0.94      0.94      0.94     15300
   macro avg       0.93      0.93      0.93     15300
weighted avg       0.94      0.94      0.94     15300



In [73]:
from joblib import dump   # Exporting tree to use in main project.
dump(dtree, 'glasses_dtree.joblib')

['glasses_dtree.joblib']